In [2]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
import pickle
import os
import playsound
import datetime

def play_done_message():
    playsound.playsound("./data/MamaDoIt.mp3")

In [13]:
# Your main code
print("Doing some work...")
geolocator = Nominatim(user_agent="NYCAccidentsProject")
total_number=int(input('How many for this go around? (multiples of 1000) '))
while total_number%1000 !=0:
    total_number=input('How many for this go around? (multiples of 1000) ')
date1=datetime.datetime.now()
for _ in range(int(total_number/1000)):
    
    pkl=[file for file in os.listdir() if file.endswith('.pkl')]
    with open(pkl[0],'rb') as f:
        all_dict=pickle.load(f)   
    initial=all_dict['last_row']
    print(initial)
    final=initial+1000
    df=all_dict['info'].iloc[initial:final]
    idx=df.index
    count=0
    for row in idx:
        #print(row)
        count+=1
        if count%1000==0:
            print(row,count)
        location = geolocator.reverse((df.loc[row, 'latitude'],df.loc[row,'longitude']), exactly_one=True)
    #print(type(all_dict['df']),type(location))
        time.sleep(0.9)
        if location:
            all_dict['df'].loc[row,'address']=location[0]
        else:
            all_dict['df'].loc[row,'address']="Nothing"

    all_dict['last_row']=len(all_dict['df'])
    print(len(all_dict['df']))
    with open(f'first{final}.pkl','wb') as f:
        pickle.dump(all_dict,f)
    
    if pkl[0]:
        os.remove(pkl[0])
        print(f"{pkl[0]} has been deleted.")
    else:
        print(f"{pkl[0]} does not exist.")
    date2=datetime.datetime.now()
    difference=round((date2-date1).total_seconds()/60,2)
    print(f'Time since last one is {difference} minutes')
    date1=date2
# Play the video when the code is done
play_done_message()

Doing some work...


How many for this go around? (multiples of 1000)  10000


206000
708499 1000
207000
first206000.pkl has been deleted.
Time since last one is 65.1 minutes
207000
711635 1000
208000
first207000.pkl has been deleted.
Time since last one is 34.94 minutes
208000
714984 1000
209000
first208000.pkl has been deleted.
Time since last one is 19.4 minutes
209000
718243 1000
210000
first209000.pkl has been deleted.
Time since last one is 19.46 minutes
210000
721488 1000
211000
first210000.pkl has been deleted.
Time since last one is 19.35 minutes
211000
724630 1000
212000
first211000.pkl has been deleted.
Time since last one is 19.43 minutes
212000
727814 1000
213000
first212000.pkl has been deleted.
Time since last one is 19.16 minutes
213000
731096 1000
214000
first213000.pkl has been deleted.
Time since last one is 19.27 minutes
214000
734286 1000
215000
first214000.pkl has been deleted.
Time since last one is 19.26 minutes
215000
737499 1000
216000
first215000.pkl has been deleted.
Time since last one is 19.24 minutes


## UNIQUE VALUES TO REVERSE GEOCODE

In [3]:
# Your main code
print("Doing some work with uniques...")
geolocator = Nominatim(user_agent="NYCAccidentsProject")
total_number=int(input('How many for this go around? (multiples of 1000) '))
while total_number%1000 !=0:
    total_number=input('How many for this go around? (multiples of 1000) ')
date1=datetime.datetime.now()
for _ in range(int(total_number/1000)):
    
    pkl=[file for file in os.listdir() if file.endswith('unique.pkl')]
    with open(pkl[0],'rb') as f:
        all_dict=pickle.load(f)   
    initial=all_dict['last_row']
    print('INITIAL :',initial)
    final=initial+1000
    df=all_dict['info'].iloc[initial:final]
    idx=df.index
    count=0
    for row in idx:
        #print(row)
        count+=1
        if count%1000==0:
            print(row,count)
        location = geolocator.reverse((df.loc[row, 'lat'],df.loc[row,'long']), exactly_one=True)
    #print(type(all_dict['df']),type(location))
        time.sleep(0.9)
        if location:
            all_dict['info'].loc[row,'address']=location[0]
        else:
            all_dict['info'].loc[row,'address']="Nothing"

    all_dict['last_row']=final
    print(final)
    with open(f'first{final}unique.pkl','wb') as f:
        pickle.dump(all_dict,f)
    
    if f'first{initial}unique.pkl' in os.listdir():
        os.remove(f'first{initial}unique.pkl')
        print(f"{f'first{initial}unique.pkl'} has been deleted.")
    else:
        print(f"{f'first{initial}unique.pkl'} does not exist.")
    date2=datetime.datetime.now()
    difference=round((date2-date1).total_seconds()/60,2)
    print(f'Time since last one is {difference} minutes')
    date1=date2
# Play the video when the code is done
play_done_message()
print('Done')

Doing some work with uniques...


How many for this go around? (multiples of 1000)  1000


INITIAL : 26000
27000
first26000unique.pkl has been deleted.
Time since last one is 5.71 minutes
Done


In [ ]:
## NEXT TASK: Create storage where to keep all locations.
## FINISHING UP AUTOMATING API CALLS


In [4]:
with open('first216000.pkl','rb') as file:
    big_one=pickle.load(file)
with open('first27000unique.pkl','rb') as file:
    small_one=pickle.load(file)
big_one_df=pd.concat([big_one['df'],big_one['info']],axis=1)
print(big_one_df[big_one_df.address.isnull()].shape)
big_one_df['location']=big_one_df['latitude'].astype(str)+big_one_df['longitude'].astype(str)
small_one_df=small_one['info']
small_one_df['location']=small_one_df['lat'].astype(str)+small_one_df['long'].astype(str)
idx_to_save=big_one_df.index
small_one_dict=small_one_df[['address','location']].set_index('location').to_dict()['address']
count=0
for idx,v in big_one_df[big_one_df.address.isnull()].iterrows():
    count+=1
    if small_one_dict.get(v['location']):
        big_one_df.loc[idx,'address']=small_one_dict.get(v['location'])
    if count%50000==0:
        print(count)
play_done_message()
print('Done')

(246052, 3)
50000
100000
150000
200000
Done


In [10]:
big_one_df.to_csv('new_locations.csv',index=False)

In [7]:
##Tryong to figure out how to fill nulls in boroughs and zip_code with values address___
big_one_df.index

Index([      6,      10,      13,      19,      21,      23,      31,      39,
            40,      41,
       ...
       2139023, 2139024, 2139027, 2139029, 2139031, 2139035, 2139037, 2139038,
       2139042, 2139045],
      dtype='int64', length=462052)

In [8]:
crashes=pd.read_csv('/Users/fcbnyc/mystuff/repos/NYCAccidents/data/Motor_Vehicle_Collisions_-_Crashes_20241204.csv')
crashes.columns=[col.lower().replace(' ','_') for col in crashes.columns]
crashes['crash_date']=pd.to_datetime(crashes.crash_date,format='%m/%d/%Y')
crashes.head()

/var/folders/ky/93l9lrgs7qgbn3j0hprgld5m0000gn/T/ipykernel_2060/1698732640.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  crashes=pd.read_csv('/Users/fcbnyc/mystuff/repos/NYCAccidents/data/Motor_Vehicle_Collisions_-_Crashes_20241204.csv')


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2021-09-11,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [54]:
for idx, v in new_crash.sample(30,random_state=111).iterrows():
    if (isinstance(v['address___'],str)) and (isninstance(v['borough'],float)):
        print(idx,v['address___'],type(v['borough']))

557153 1309, Forest Avenue, Westerleigh, Staten Island, Richmond County, City of New York, New York, 10302, United States <class 'float'>


In [57]:
new_crash.loc[557153,'borough']

nan

In [78]:
# for value in new_crash.address___.sample(30,random_state=111):
#     print(value)
values_options=set([len(value.split(',')) for value in new_crash.address___ if isinstance(value,str)])
print(values_options)
for v in values_options:
    list_=[value for value in new_crash.address___ if isinstance(value,str) and len(value)==v]
    if len(list_)==0:
        print(v, len(list_))
    else:
        print(v,[value for value in new_crash.address___ if isinstance(value,str) and len(value)==v][0])
[v for v in new_crash]

{1, 6, 7, 8, 9, 10, 11, 12, 13, 14}
1 0
6 0
7 Nothing
8 0
9 0
10 0
11 0
12 0
13 0
14 0


In [34]:
#big_one_df.columns=[col+'_' for col in big_one_df.columns]
new_crash=pd.concat([crashes,big_one_df['address___']],axis=1)
new_crash.to_csv('new_crashes_fixed.csv',index=False)

In [32]:
for idx,v in new_crash.sample(30).iterrows():
    print(v['location'],v['location__'])

(40.6129575, -73.9590769) nan
(40.705154, -73.85597) 40.705154-73.85597
(40.8396059, -73.9181388) nan
(40.86832, -73.8331589) nan
(40.70705, -73.93209) nan
(40.5946211, -73.9974189) 40.5946211-73.9974189
(40.66612, -73.82225) nan
(40.678394, -73.94692) nan
(40.688236, -73.76196) nan
nan nan
(40.7453835, -73.9947093) nan
(40.7270902, -73.9766387) nan
(40.7460428, -73.8260812) nan
(40.736214, -73.713486) nan
(40.71285, -73.93457) nan
(40.683723, -73.96797) 40.683723-73.96797
(40.86749, -73.840515) nan
(40.7598833, -73.9367906) nan
(40.59406, -73.7891194) nan
(40.7057388, -73.9446867) nan
(40.701645, -73.73046) nan
(40.8658679, -73.8462386) nan
(40.81719, -73.93446) nan
(40.7879013, -73.9536717) nan
(40.699405, -73.953369) nan
(40.774853, -73.98067) 40.774853-73.98067
(40.7707422, -73.9945485) nan
(40.6250313, -73.929483) nan
(40.7692226, -73.967306) nan
(40.6312149, -74.1005091) nan


In [24]:
nulls_in_boroughs=new_crash[(new_crash.borough.isnull()) & (new_crash.latitude_.notnull())].shape[0]
print(f"These are the number of nulls in borough but that I have lat and long: {nulls_in_boroughs}")

These are the number of nulls in borough but that I have lat and long: 462052


In [13]:
uniques_df=pd.DataFrame()
uniques_df['lat']=[x.split(',')[0].strip() for x in TO_DO]
uniques_df['long']=[x.split(',')[1].strip() for x in TO_DO]
uniques_df=uniques_df.iloc[1:]
uniques_df['address']='to_fill'
uniques_df

NameError: name 'TO_DO' is not defined

In [55]:
mydict.keys()

dict_keys(['df', 'last_row', 'info'])

In [89]:
new_dict={}
new_dict['last_row']=0
new_dict['info']=uniques_df
with open('final0unique.pkl','wb') as file:
    pickle.dump(new_dict,file)

In [89]:
with open('final0unique.pkl','rb') as file:
    test=pickle.load(file)
test

{'last_row': 0,
 'info':              lat        long  address
 1      40.608757  -74.038086  to_fill
 2        40.8047   -73.91243  to_fill
 3      40.798256   -73.82744  to_fill
 4       40.77077   -73.91727  to_fill
 5       40.83801   -73.87329  to_fill
 ...          ...         ...      ...
 57930   40.63617  -73.928734  to_fill
 57931   40.68361   -73.91172  to_fill
 57932  40.659843   -73.94316  to_fill
 57933  40.552235   -74.21214  to_fill
 57934   40.83305   -73.90592  to_fill
 
 [57934 rows x 3 columns]}

,lat,long,address
1,40.608757,-74.038086,carrer Toledo
2,40.8047,-73.91243,to_fill
3,40.798256,-73.82744,to_fill
4,40.77077,-73.91727,to_fill
5,40.83801,-73.87329,to_fill
...,...,...,...
57930,40.63617,-73.928734,to_fill
57931,40.68361,-73.91172,to_fill
57932,40.659843,-73.94316,to_fill
57933,40.552235,-74.21214,to_fill
